# B3 Derivatives

## Install Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

## Future Contracts

#### Link

https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/derivativos/resumo-estatistico/sistema-pregao/

#### Function

In [2]:
def futures_data(ticker,start_date,end_date):

    # Dict tickers
    dict_tickers = {
        # most liquid contracts
        "DI1":17,
        "DOL":15,
        "IND":16,
        "ISP":15,
        "WIN":16,
        "WDO":15,
        "WSP":15,
        "XFI":15,

        # commodities future contracts
        "BGI":15,
        "CCM":15,
        "ICF":15,
        "SJC":15,

        # currency future contracts
        "AFS":15,
        "ARB":15,
        "ARS":15,
        "AUD":15,
        "AUS":15,
        "CAD":15,
        "CAN":15,
        "CHF":15,
        "CHL":15,
        "CLP":15,
        "CNH":15,
        "CNY":15,

        # term contracts
        "ABEVO":15,
        "B3SAO":15,
        "BBASO":15,
        "BBDCP":15,
        "CCROO":15,
        "CIELO":15,
        "CMIGP":15,
        "COGNO":15,
        "CSNAO":15,
        "ELETO":15,
        "GGBRP":15,
        "HYPEO":15,
        "ITSAP":15,
        "ITUBP":15,
        "LRENO":15,
        "MGLUO":15,
        "NTCOO":15,
        "PCARO":15,
        "PETRP":15,
        "PSSAO":15,
        "RENTO":15,
        "SUZBO":15,
        "USIMA":15,
        "VIIAO":15}

    # Dict cols: different-length columns from ticker to ticker
    dict_cols = {15:["date",
                    "ticker",
                    "contract_position",
                    "maturity",
                    "opening_contracts",
                    "closing_contracts",
                    "trading_number",
                    "trading_contracts",
                    "volume",
                    "open",
                    "low",
                    "high",
                    "mean",
                    "close",
                    "adjustment",
                    "points_delta",
                    "last_buy",
                    "last_sell"],
                16:["date",
                    "ticker",
                    "contract_position",
                    "maturity",
                    "opening_contracts",
                    "closing_contracts",
                    "trading_number",
                    "trading_contracts",
                    "volume",
                    "oscilation",
                    "open",
                    "low",
                    "high",
                    "mean",
                    "close",
                    "adjustment",
                    "points_delta",
                    "last_buy",
                    "last_sell"],
                17:["date",
                    "ticker",
                    "contract_position",
                    "maturity",
                    "opening_contracts",
                    "closing_contracts",
                    "trading_number",
                    "trading_contracts",
                    "volume",
                    "previous_adjustment",
                    "corrected_previous_adjustment",
                    "open",
                    "low",
                    "high",
                    "mean",
                    "close",
                    "adjustment",
                    "points_delta",
                    "last_buy",
                    "last_sell"]
                    }

    # Dates
    dates = list()
    date = dt.date(int(start_date[:4]),int(start_date[-5:-3]),int(start_date[-2:]))
    while date <= dt.date(int(end_date[:4]),int(end_date[-5:-3]),int(end_date[-2:])):
        if date.weekday()!=5 and date.weekday()!=6: 
            dates.append(date.strftime('%Y-%m-%d'))
        date += dt.timedelta(1)

    # Contracts
    months = ["F","G","H","J","K","M","N","Q","U","V","X","Z"]
    years = [str(year)[-2:] for year in range(int(start_date[:4]),int(end_date[:4])+7)]
    combine_list = [month + year for month in months for year in years]

    # Values
    numbers_list = [".",",","0","1","2","3","4","5","6","7","8","9"]

    # Output object
    df_results = pd.DataFrame(columns = dict_cols[dict_tickers[ticker]])

    # Iteration Object
    dict_results = dict()

    # For each date
    for date in tqdm(dates):

        # Check request from a date
        url = f"https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-sistema-pregao-ptBR.asp?Data={date[-2:]}/{date[-5:-3]}/{date[:4]}&Mercadoria={ticker}"
        request = requests.get(url)

        if request.status_code == 200:
            
            try:
                # Get and process data
                html_soup = BeautifulSoup(request.text, 'html.parser')
                string_soup = str(list(html_soup)[-2])
                list_soup = string_soup.split(";")

                # For each cotract (row)
                cont = 0
                contracts_list = list()
                for string in list_soup:

                    # ticker columns
                    if cont % dict_tickers[ticker] == 0:
                        sub_list = string.split(">")
                        for enum,sub_string in enumerate(sub_list):
                            if sub_string[:3] in combine_list:
                                values_list = list()
                                values_list.append(sub_string[:3])
                                cont += 1
                    else:
                        sub_list = string.split(">")
                        for enum,sub_string in enumerate(sub_list):
                            if sub_string[0] in numbers_list:
                                number = str()
                                i = 0
                                while sub_string[i] in numbers_list:
                                    number += sub_string[i]
                                    i += 1
                                values_list.append(number)
                                cont += 1

                                # Store contract (row)
                                if len(values_list) == 15:
                                    contracts_list.append(values_list)

                # Store contracts
                if contracts_list != list():
                    dict_results[f"{date}"] = contracts_list

            except:
                print(f"\n Error in {date}")
                continue

        else:
            print(f"\n Error in connection")
            return
            
    # Store into output object
    for i,key in enumerate(dict_results.keys()):
        for j,contract_list in enumerate(dict_results[key]):
            df_results.loc[i*15 + j] = [key,ticker,j+1] + contract_list 

    return df_results

In [3]:
ticker = "WDO"
start_date = "2022-01-01"
end_date = "2022-12-31"
df_results = futures_data(ticker,start_date,end_date)
df_results

100%|████████████████████████████████████████████████████████████████████████████████| 260/260 [01:49<00:00,  2.37it/s]


,date,ticker,contract_position,maturity,opening_contracts,closing_contracts,trading_number,trading_contracts,volume,open,low,high,mean,close,adjustment,points_delta,last_buy,last_sell
0,2022-01-03,WDO,1,F22,777.750,777.750,0,0,0,"0,000","0,000","0,000","0,000","0,000","5.580,5000","0,0000","0,000","0,000"
1,2022-01-03,WDO,2,G22,379.888,402.117,772.997,2.352.883,133.365.541.545,"5.609,500","5.595,000","5.729,000","5.668,175","5.727,000","5.698,9330","82,0090","5.727,000","5.727,500"
2,2022-01-03,WDO,3,H22,31,100,36,91,5.198.635,"5.670,000","5.640,000","5.760,500","5.712,785","5.760,500","5.739,5210","82,8990","5.762,000","5.768,000"
3,2022-01-03,WDO,4,J22,503,508,2,6,347.425,"5.790,500","5.790,000","5.790,500","5.790,416","5.790,000","5.790,4260","83,7960","0,000","0,000"
4,2022-01-03,WDO,5,K22,0,0,0,0,0,"0,000","0,000","0,000","0,000","0,000","5.836,1700","84,4890","0,000","0,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,2022-12-29,WDO,7,N23,0,0,0,0,0,"0,000","0,000","0,000","0,000","0,000","5.475,3770","0,0000","0,000","0,000"
3742,2022-12-29,WDO,8,V23,39,39,0,0,0,"0,000","0,000","0,000","0,000","0,000","5.567,2770","44,9750","0,000","0,000"
3743,2022-12-29,WDO,9,F24,0,0,0,0,0,"0,000","0,000","0,000","0,000","0,000","5.643,2190","43,4750","0,000","0,000"
3744,2022-12-29,WDO,10,V24,78,78,0,0,0,"0,000","0,000","0,000","0,000","0,000","5.919,0890","40,5710","0,000","0,000"
